# ODMR Optimiser

Trigger a refocus after each ODMR line.

This notebook:

- Starts ODMR scan
- Immediately requests to stop it
- Waits for it to actually stop
- Performs an optimization and waits for it to finish
- Restarts ODMR scan

Reconfigure the first cell to have the correct names for the ODMR logic and optimiser logic modules.

Note we run `odmr_optimise` in a thread so we can stop it - Qudi currently doesn't respond to the normal Jupyter 'stop' button! :)

In [ ]:
import time
import threading

_optimizer_logic = optimizerlogic
_odmr_logic = odmrlogic
_poimanager_logic = poimanagerlogic

stop_flag = threading.Event()

# Optimise every..
optimisation_time = 120 #seconds (must be whole number)

In [ ]:
def odmr_optimise():
    print('Starting ODMR optimiser')
    while not stop_flag.is_set():
        # Start ODMR
        _odmr_logic.start_odmr_scan()
        
        # Wait for the time we want between optimisations
        for i in range(optimisation_time):
            time.sleep(1)
            if stop_flag.is_set():
                return

        # Request to stop ODMR
        _odmr_logic.stop_odmr_scan()

        # Wait until it actually stops
        while _odmr_logic.module_state() == 'locked':
            time.sleep(0.1)
            if stop_flag.is_set():
                return

        # Request optimization from POI manager
        _poimanager_logic.optimise_poi_position()
        
        # Wait for it to start
        time.sleep(1)

        # Wait for optimisation to finish
        while _optimizer_logic.module_state() == 'locked':
            time.sleep(0.1)
            if stop_flag.is_set():
                return

In [ ]:
# Start the routine
stop_flag.clear()
threading.Thread(target=odmr_optimise).start()

In [ ]:
# Request to stop the routine
stop_flag.set()